# Building Kaggle Database and Queries

**Table of Contents:**

- Introduction
- Building the database
- Simple queries with SQL and the Session
    - The second user
    - Organizations with most users
    - Users with most team memberships
- More complex queries
    - The first forum message for the top 10 forum messengers
    - Which forum topics get the most votes from the 20 percent most frequent forum messengers?
    - Top 10 users with longest time intervals regarding forum message posts
- Conclusion

## Introduction

Kaggle is a data science platform that includes competitions, datasets, kernels, forums, etc. Kaggle stores the metadata of these services into seperate csv files. These csv files can be downloaded from [here](https://www.kaggle.com/kaggle/meta-kaggle). In this project I will use these separate csv files to build a relational database, and then I will build some select queries.

In order to build a database I use SQLAlchemy, which is a library that facilitates the communication between Python programs and databases. I will also use SQLAlchemy to set up some simple queries. However, for more complex select queries I will use SQL language.

Let us import the dependencies first.

In [1]:
import os
os.chdir("d:/Projects/Meta Kaggle/data")

In [2]:
# import dependencies

import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Numeric, String, Date, DateTime, Boolean, Text, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import inspect
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy import func
from sqlalchemy import asc, desc

## Building the database

In order to build the database I will use SQLAlchemy ORM instead of SQLAlchemy Core. I define the database schema by setting up classes, which include info on the table name, attribute type, attribute size, primary key, foreign keys etc. These represent the CREATE TABLE statements in normal SQL language. I also will define relationships (including backref), as these come in handy when setting up select queries wit the Session later on.

In [3]:
# When defining the Schema, we will inherit from the declaritave_base object
Base = declarative_base()

In [4]:
# For each dataset define the schema, and include relationships

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer(), primary_key=True)
    username = Column(String(25))
    displayname = Column(String(25))
    registerdate = Column(Date())
    performancetier = Column(Integer())

class Organization(Base):
    __tablename__ = "organizations"
    
    id = Column(Integer(), primary_key=True)
    name = Column(String(50))
    slug = Column(String(40))
    creationdate = Column(Date())
    description = Column(Text())    

class UserOrganization(Base):
    __tablename__ = "userorganizations"
    
    id = Column(Integer(), primary_key=True)
    userid = Column(Integer(), ForeignKey("users.id"))
    organizationid = Column(Integer(), ForeignKey("organizations.id"))
    joindate = Column(Date())
    
    user = relationship("User", backref=backref("userorganizations"))
    organization = relationship("Organization", backref=backref("userorganizations"))


class UserAchievement(Base):
    __tablename__ = "userachievements"
    
    id = Column(Integer(), primary_key=True)
    userid = Column(Integer(), ForeignKey("users.id"))
    achievementtype = Column(String(12))
    tier = Column(Integer())
    tierachievementdate = Column(Date())
    points = Column(Integer())
    currentranking = Column(Integer())
    highestranking = Column(Integer())
    totalgold = Column(Integer())
    totalsilver = Column(Integer())
    totalbronze = Column(Integer())
    
    user = relationship("User", backref=backref("userachievements"))
    
class UserFollower(Base):
    __tablename__ = "userfollowers"
    
    id = Column(Integer(), primary_key=True)
    userid = Column(Integer(), ForeignKey("users.id"))
    followinguserid = Column(Integer())
    creationdate = Column(Date())
    
    user = relationship("User", backref=backref("userfollowers"))
    
class Team(Base):
    __tablename__ = "teams"
    
    id = Column(Integer(), primary_key=True)
    competitionid = Column(Integer())
    teamleaderid = Column(Integer(), ForeignKey("users.id"))
    teamname = Column(String(25))
    scorefirstsubmitteddate = Column(Date())
    lastsubmissiondate = Column(Date())
    publicleaderboardsubmissionid = Column(Integer())
    privateleaderboardsubmissionid = Column(Integer())
    isbenchmark = Column(Boolean())
    medal = Column(Integer())
    medalawarddate = Column(Date())
    publicleaderboardrank = Column(Integer())
    privateleaderboardrank = Column(Integer())
    
    teamleader = relationship("User", backref=backref("teamleaders"))
    
class TeamMembership(Base):
    __tablename__ = "teammemberships"
    
    id = Column(Integer(), primary_key=True)
    teamid = Column(Integer(), ForeignKey("teams.id"))
    userid = Column(Integer(), ForeignKey("users.id"))
    requestdate = Column(Date())
    
    user = relationship("User", backref=backref("teammemberships"))
    team = relationship("Team", backref=backref("teammemberships"))

class Tag(Base):
    __tablename__ = "tags"
    
    id = Column(Integer(), primary_key=True)
    parenttagid = Column(Integer())
    name = Column(String(40))
    slug = Column(String(40))
    fullpath = Column(String(90))
    description = Column(Text())
    datasetcount = Column(Integer())
    competitioncount = Column(Integer())
    kernelcount = Column(Integer())    

class Forum(Base):
    __tablename__ = "forums"
    
    id = Column(Integer(), primary_key=True)
    parentforumid = Column(Integer())
    title = Column(String(100))    

class ForumTopic(Base):
    __tablename__ = "forumtopics"
    
    id = Column(Integer(), primary_key=True)
    forumid = Column(Integer(), ForeignKey("forums.id"))
    kernelid = Column(Integer())
    lastforummessageid = Column(Integer())
    firstforummessageid = Column(Integer())
    creationdate = Column(DateTime())
    lastcommentdate = Column(DateTime())
    title = Column(String(255))
    issticky = Column(Boolean())
    totalviews = Column(Integer())
    score = Column(Integer())
    totalmessages = Column(Integer())
    totalreplies = Column(Integer())
    
    forum = relationship("Forum", backref=backref("forumtopics"))

class ForumMessage(Base):
    __tablename__ = "forummessages"
    
    id = Column(Integer(), primary_key=True)
    forumtopicid = Column(Integer(), ForeignKey("forumtopics.id"))
    postuserid = Column(Integer(), ForeignKey("users.id"))
    postdate = Column(DateTime())
    replytoforummessageid = Column(Integer())
    message = Column(Text())
    medal = Column(Integer())
    medalawarddate = Column(Date())
    
    forumtopic = relationship("ForumTopic", backref=backref("forummessages"))
    postuser = relationship("User", backref=backref("forummessages"))

class ForumMessageVote(Base):
    __tablename__ = "forummessagevotes"
    
    id = Column(Integer(), primary_key=True)
    forummessageid = Column(Integer(), ForeignKey("forummessages.id"))
    fromuserid = Column(Integer(), ForeignKey("users.id"))
    touserid = Column(Integer())
    votedate = Column(Date())
    
    forummessage = relationship("ForumMessage", backref=backref("forummessagevotes"))
    fromuser = relationship("User", backref=backref("forummessagevotes"))
                   
class Dataset(Base):
    __tablename__ = "datasets"
    
    id = Column(Integer(), primary_key=True)
    creatoruserid = Column(Integer(), ForeignKey("users.id"))
    owneruserid = Column(Integer())
    ownerorganizationid = Column(Integer())
    currentdatasetversionid = Column(Integer())
    currentdatasourceversionid = Column(Integer())
    forumid = Column(Integer(), ForeignKey("forums.id"))
    type = Column(Integer())
    creationdate = Column(DateTime())
    reviewdate = Column(Date())
    featuredate = Column(Date())
    lastactivitydate = Column(Date())
    totalviews = Column(Integer())
    totaldownloads = Column(Integer())
    totalvotes = Column(Integer())
    totalkernels = Column(Integer())
    
    creatoruser = relationship("User", backref=backref("datasets_created"))
    forum = relationship("Forum", backref=backref("datasets"))

class Datasource(Base):
    __tablename__ = "datasources"
    
    id = Column(Integer(), primary_key=True)
    creatoruserid = Column(Integer(), ForeignKey("users.id"))
    creationdate = Column(DateTime())
    type = Column(Integer())
    currentdatasourceversionid = Column(Integer())  
    
    creatoruser = relationship("User", backref=backref("datasources_created"))
    
class DatasetVersion(Base):
    __tablename__ = "datasetversions"
    
    id = Column(Integer(), primary_key=True)
    datasetid = Column(Integer(), ForeignKey("datasets.id"))
    datasourceversionid = Column(Integer(), ForeignKey("datasources.id"))
    creatoruserid = Column(Integer(), ForeignKey("users.id"))
    licensename = Column(String(75))
    creationdate = Column(DateTime())
    versionnumber = Column(Integer())
    title = Column(String(60))
    slug = Column(String(50))
    subtitle = Column(Text())
    description = Column(Text())
    versionnotes = Column(Text())
    totalcompressedbytes = Column(Integer())
    totaluncompressedbytes = Column(Integer())
    
    dataset = relationship("Dataset", backref=backref("datasetversions"))
    datasource = relationship("Datasource", backref=backref("datasetversions"))
    creatoruser = relationship("User", backref=backref("datasetversions_created"))

class DatasetVote(Base):
    __tablename__ = "datasetvotes"
    
    id = Column(Integer(), primary_key=True)
    userid = Column(Integer(), ForeignKey("users.id"))
    datasetversionid = Column(Integer(), ForeignKey("datasetversions.id"))
    votedate = Column(Date())
    
    user = relationship("User", backref=backref("datasetvotes"))
    datasetversion = relationship("DatasetVersion", backref=backref("datasetvotes"))
    
class DatasetTag(Base):
    __tablename__ = "datasettags"
    
    id = Column(Integer(), primary_key=True)
    datasetid = Column(Integer(), ForeignKey("datasets.id"))
    tagid = Column(Integer(), ForeignKey("tags.id"))
    
    dataset = relationship("Dataset", backref=backref("datasettags"))
    tag = relationship("Tag", backref=backref("datasettags"))

class KernelVersion(Base):
    __tablename__ = "kernelversions"
    
    id = Column(Integer(), primary_key=True)
    scriptid = Column(Integer())
    parentscriptversionid = Column(Integer())
    scriptlanguageid = Column(Integer())
    authoruserid = Column(Integer(), ForeignKey("users.id"))
    creationdate = Column(DateTime())
    versionnumber = Column(Integer())
    title = Column(String(110))
    evaluationdate = Column(Date())
    ischange = Column(Boolean())
    totallines = Column(Integer())
    linesinsertedfromprevious = Column(Integer())
    lineschangedfromprevious = Column(Integer())
    linesunchangedfromprevious = Column(Integer())
    linesinsertedfromfork = Column(Integer())
    linesdeletedfromfork = Column(Integer())
    lineschangedfromfork = Column(Integer())
    linesunchangedfromfork = Column(Integer())
    totalvotes = Column(Integer())
    
    authoruser = relationship("User", backref=backref("kernelversions_authored"))
    
class Kernel(Base):
    __tablename__ = "kernels"
    
    id = Column(Integer(), primary_key=True)
    authoruserid = Column(Integer(), ForeignKey("users.id"))
    currentkernelversionid = Column(Integer())
    forkparentkernelversionid = Column(Integer())
    forumtopicid = Column(Integer())
    firstkernelversionid = Column(Integer())
    creationdate = Column(DateTime())
    evaluationdate = Column(Date())
    madepublicdate = Column(Date())
    isprojectlanguagetemplate = Column(Boolean())
    currenturlslug = Column(String(65))
    medal = Column(Integer())
    medalawarddate = Column(Date())
    totalviews = Column(Integer())
    totalcomments = Column(Integer())
    totalvotes = Column(Integer())
    
    authoruser = relationship("User", backref=backref("kernels_authored"))

class KernelVote(Base):
    __tablename__ = "kernelvotes"
    
    id = Column(Integer(), primary_key=True)
    userid = Column(Integer(), ForeignKey("users.id"))
    kernelversionid = Column(Integer(), ForeignKey("kernelversions.id"))
    votedate = Column(Date())
    
    user = relationship("User", backref=backref("kernelvotes"))
    kernelversion = relationship("KernelVersion", backref=backref("kernelvotes"))

class KernelTag(Base):
    __tablename__ = "kernaltags"
    
    id = Column(Integer(), primary_key=True)
    kernelid = Column(Integer(), ForeignKey("kernels.id"))
    tagid = Column(Integer(), ForeignKey("tags.id"))
    
    kernel = relationship("Kernel", backref=backref("kerneltags"))
    tag = relationship("Tag", backref=backref("kerneltags"))

class Submission(Base):
    __tablename__ = "submissions"
    
    id = Column(Integer(), primary_key=True)
    submitteduserid = Column(Integer(), ForeignKey("users.id"))
    teamid = Column(Integer(), ForeignKey("teams.id"))
    sourcekernelversionid = Column(Integer())
    submissiondate = Column(Date())
    scoredate = Column(Date())
    isafterdeadline = Column(Boolean())
    publicscoreleaderboarddisplay = Column(Float())
    publicscorefullprecision = Column(Float())
    privatescoreleaderboarddisplay = Column(Float())
    privatescorefullprecision = Column(Float())
    
    submitteduser = relationship("User", backref=backref("submissions"))
    team = relationship("Team", backref=backref("submissions"))
                   
class Competition(Base):
    __tablename__ = "competitions"
    
    id = Column(Integer(), primary_key=True)
    slug = Column(String(80))
    title = Column(String(100))
    subtitle = Column(Text())
    hostsegmenttitle = Column(String(16))
    forumid = Column(Integer())
    organizationid = Column(Integer())
    competitiontypeid = Column(Integer())
    hostname = Column(String(85))
    enableddate = Column(DateTime())
    deadlinedate = Column(DateTime())
    prohibitnewentrantsdeadlinedate = Column(DateTime())
    teammergerdeadlinedate = Column(DateTime())
    teammodeldeadlinedate = Column(DateTime())
    modelsubmissiondeadlinedate = Column(DateTime())
    finalleaderboardhasbeenverified = Column(Boolean())
    haskernels = Column(Boolean())
    onlyallowkernelsubmissions = Column(Boolean())
    hasleaderboard = Column(Boolean())
    leaderboardpercentage = Column(Integer())
    leaderboarddisplayformat = Column(Integer())
    evaluationalgorithmabbreviation = Column(String(60))
    evaluationalgorithmname = Column(String(70))
    evaluationalgorithmdescription = Column(Text())
    evaluationalgorithmismax = Column(Boolean())
    validationsetname = Column(String(45))
    validationsetvalue = Column(String(40))
    maxdailysubmissions = Column(Integer())
    numscoredsubmissions = Column(Integer())
    maxteamsize = Column(Integer())
    banteammergers = Column(Boolean())
    enableteammodels = Column(Boolean())
    enablesubmissionmodelhashes = Column(Boolean())
    enablesubmissionmodelattachments = Column(Boolean())
    rewardtype = Column(String(10))
    rewardquantity = Column(Integer())
    numprizes = Column(Integer())
    userrankmultiplier = Column(Float())
    canqualifytiers = Column(Boolean())
    totalteams = Column(Integer())
    totalcompetitors = Column(Integer())
    totalsubmissions = Column(Integer())
    
    
class CompetitionTag(Base):
    __tablename__ = "competitiontags"
    
    id = Column(Integer(), primary_key=True)
    competitionid = Column(Integer(), ForeignKey("competitions.id"))
    tagid = Column(Integer(), ForeignKey("tags.id"))
    
    competition = relationship("Competition", backref=backref("competitiontags"))
    tag = relationship("Tag", backref=backref("competitiontags"))
    
class KernelVersionCompetitionSource(Base):
    __tablename__ = "kernelversioncompetitionsources"
    
    id = Column(Integer(), primary_key=True)
    kernelversionid = Column(Integer(), ForeignKey("kernelversions.id"))
    sourcecompetitionid = Column(Integer(), ForeignKey("competitions.id"))
    
    kernelversion = relationship("KernelVersion", backref=backref("kernelversioncompetitionsources"))
    sourcecompetition = relationship("Competition", backref=backref("kernelversioncompetitionsources"))
    
class KernelVersionDatasetSource(Base):
    __tablename__ = "kernelversiondatasetsources"
    
    id = Column(Integer(), primary_key=True)
    kernelversionid = Column(Integer(), ForeignKey("kernelversions.id"))
    sourcedatasetversionid = Column(Integer(), ForeignKey("datasetversions.id"))
    
    kernelversion = relationship("KernelVersion", backref=backref("kernelversiondatasetsources"))
    sourcedatasetversion = relationship("DatasetVersion", backref=backref("kernelversiondatasetsources"))

class KernelVersionKernelSource(Base):
    __tablename__ = "kernelversionkernelsources"
    
    id = Column(Integer(), primary_key=True)
    kernelversionid = Column(Integer(), ForeignKey("kernelversions.id"))
    sourcekernelversionid = Column(Integer())
    
    kernelversion = relationship("KernelVersion", backref=backref("kernelversionkernelsources"))


Now that we have defined the database schema, we are going to create the database tables with an instance of an engine.

In [5]:
# Create instance of an engine with sqlite
engine = create_engine("sqlite:///kaggle_database.db")

In [6]:
# Create database tables
Base.metadata.create_all(engine)

Let's inspect whether all the tables that were defined above are now in the database:

In [7]:
# Inspect the database
inspector = inspect(engine)
print(inspector.get_table_names())

['competitions', 'competitiontags', 'datasets', 'datasettags', 'datasetversions', 'datasetvotes', 'datasources', 'forummessages', 'forummessagevotes', 'forums', 'forumtopics', 'kernaltags', 'kernels', 'kernelversioncompetitionsources', 'kernelversiondatasetsources', 'kernelversionkernelsources', 'kernelversions', 'kernelvotes', 'organizations', 'submissions', 'tags', 'teammemberships', 'teams', 'userachievements', 'userfollowers', 'userorganizations', 'users']


As you can see, all the tables are in the database. Now it is time to populate the database with the data. For this I am going to do a simple extract, transform, and load (etl) procedure with the individual csv files. 

First I set up a list with tuples, where each tuple contains some metadata on the dataset. More specifically, the etl procedure needs to handle date formats well, and therefore data variables are specified in this tuple.

Then I will use the pandas to_sql function to populate all the databases.

In [8]:
# Create a list with tuples, where each tuple contains the relevant info for the etl procedure

etl_info = [("Users.csv", "users", ["RegisterDate"]),
            ("Organizations.csv", "organizations", ["CreationDate"]),
            ("UserOrganizations.csv", "userorganizations", ["JoinDate"]),    
            ("UserAchievements.csv", "userachievements", ["TierAchievementDate"]),
            ("UserFollowers.csv", "userfollowers", ["CreationDate"]),
            ("Teams.csv", "teams", ["ScoreFirstSubmittedDate", "LastSubmissionDate", "MedalAwardDate"]),
            ("TeamMemberships.csv", "teammemberships", ["RequestDate"]),
            ("Tags.csv", "tags", []),
            ("Forums.csv", "forums", []),
            ("ForumTopics.csv", "forumtopics", ["CreationDate", "LastCommentDate"]),
            ("ForumMessages.csv", "forummessages", ["PostDate", "MedalAwardDate"]),
            ("ForumMessageVotes.csv", "forummessagevotes", ["VoteDate"]),
            ("Datasets.csv", "datasets", ["CreationDate", "ReviewDate", "FeatureDate", "LastActivityDate"]),
            ("Datasources.csv", "datasources", ["CreationDate"]),
            ("DatasetVersions.csv", "datasetversions", ["CreationDate"]),
            ("DatasetVotes.csv", "datasetvotes", ["VoteDate"]),
            ("DatasetTags.csv", "datasettags", []),
            ("KernelVersions.csv", "kernelversions", ["CreationDate", "EvaluationDate"]),
            ("Kernels.csv", "kernels", ["CreationDate", "EvaluationDate", "MadePublicDate", "MedalAwardDate"]),
            ("KernelVotes.csv", "kernelvotes", ["VoteDate"]),
            ("KernelTags.csv", "kerneltags", []),
            ("Submissions.csv", "submissions", ["SubmissionDate", "ScoreDate"]),
            ("Competitions.csv", "competitions", ["EnabledDate","DeadlineDate", "ProhibitNewEntrantsDeadlineDate", "TeamMergerDeadlineDate", "TeamModelDeadlineDate", "ModelSubmissionDeadlineDate"]),
            ("CompetitionTags.csv", "competitiontags", []),
            ("KernelVersionCompetitionSources.csv", "kernelversioncompetitionsources", []),
            ("KernelVersionDatasetSources.csv", "kernelversiondatasetsources", []),
            ("KernelVersionKernelSources.csv", "kernelversionkernelsources", [])
           ]
           


In [9]:
# Extract, transform and load
for dataset in etl_info:
    if len(dataset[2])==0:
        data = pd.read_csv(dataset[0])
    else:
        data = pd.read_csv(dataset[0], parse_dates=dataset[2], dayfirst=False)
    
    data.to_sql(dataset[1], engine, if_exists="append", index=False, chunksize=500)

# Simple queries with SQL and the Session

In this section I am going to build some simple queries using two methods:
-    Using normal SQL language and queries
-    Using the SQLAlchemy ORM Session

Let's instantiate the Session first and obtain some data on the second user in the user dataset.

In [10]:
Session = sessionmaker(bind=engine)
session = Session()

### The second user

First, we are going to collect some data on the second Kaggle user. I will only use the Session for this, but similar SQL queries could of course easily be built with a WHERE clause.

In [11]:
# get the second user
user2 = session.query(User)[1]
print(user2.id, user2.username)

368 antgoldbloom


In [12]:
# print some data with respect to the second user:

# organizations
print("User 2 is linked to {} organizations".format(len(user2.userorganizations)))
# teams
print("User 2 has led {} teams".format(len(user2.teamleaders)))
print("User 2 has joined {} teams".format(len(user2.teammemberships)))
# datasets
print("User 2 has created {} datasets".format(len(user2.datasets_created)))
print("User 2 has voted on {} datasets".format(len(user2.datasetvotes)))
# kernels
print("User 2 has authored {} kernels".format(len(user2.kernels_authored)))
# submissions
print("User 2 has made {} submissions".format(len(user2.submissions)))
# forum messages
print("User 2 has posted {} forum messages".format(len(user2.forummessages)))

User 2 is linked to 3 organizations
User 2 has led 91 teams
User 2 has joined 92 teams
User 2 has created 21 datasets
User 2 has voted on 0 datasets
User 2 has authored 186 kernels
User 2 has made 78 submissions
User 2 has posted 571 forum messages


In [13]:
# the organizations that are attached to this user are:
for i, assoc in enumerate(user2.userorganizations):
    print(i, assoc.organization.name)

0 Kaggle
1 Hacker News
2 Allen Institute For AI


As you can see, it is quite easy to obtain data on a specific user by using the Session. Let's now build some more complex queries using the two methods:
-    Using normal SQL language and queries
-    Using the Session

### Organizations with most users

Let's assume that Kaggle wants to proactively expand its user base by targetting specific organizations. They are therefore interested in which organizations have the most users. Let's set up queries, first with normal SQL language, and then with the Session.

In [14]:
# SQL language: organizations with most users
query = """SELECT organizations.id, organizations.name, COUNT(users.id) AS "number of users"
            FROM organizations
            INNER JOIN userorganizations ON userorganizations.organizationid=organizations.id
            INNER JOIN users ON users.id=userorganizations.userid
            GROUP BY organizations.id
            ORDER BY "number of users" DESC
            LIMIT 10;

"""
retrieved = pd.read_sql(query, engine)
retrieved

,id,name,number of users
0,642,OpenDataScience [ods.ai],175
1,1776,Petrobras,38
2,1304,Google BigQuery,36
3,2034,Texas A&M University,29
4,4,Kaggle,29
5,2137,Census TIGER/Line,28
6,1403,Leeds Business Analytics,27
7,185,US Bureau of Labor Statistics,25
8,540,Federal Reserve,24
9,2240,Cursos TIC,23


In [15]:
# The Session: organizations with most users 
result = session.query(Organization.id, Organization.name, func.count(User.username).label("number of users")).filter((Organization.id==UserOrganization.organizationid)&(UserOrganization.userid==User.id)).group_by(Organization.id).order_by(desc("number of users")).limit(10).all()
print(result)

[(642, 'OpenDataScience [ods.ai]', 175), (1776, 'Petrobras', 38), (1304, 'Google BigQuery', 36), (2034, 'Texas A&M University', 29), (4, 'Kaggle', 29), (2137, 'Census TIGER/Line', 28), (1403, 'Leeds Business Analytics', 27), (185, 'US Bureau of Labor Statistics', 25), (540, 'Federal Reserve', 24), (2240, 'Cursos TIC', 23)]


As you can see, both methods yield the same results. OpenDataScience is the clear winner.

### Users with most team memberships

Now we are interested in the most active users. We want to know which of the users have the most team memberships:

In [16]:
# SQL language: users with most team memberships
query = """SELECT users.id, users.username, COUNT(teammemberships.id) AS "number of team memberships"
            FROM users
            INNER JOIN teammemberships ON users.id=teammemberships.userid
            GROUP BY users.id
            ORDER BY "number of team memberships" DESC
            LIMIT 10;
"""
retrieved = pd.read_sql(query, engine)
retrieved

,id,username,number of team memberships
0,5030586,wchenfla,1651
1,929585,plarmuseau,823
2,3258,wcukierski,534
3,993,benhamner,476
4,18463,mathurinache,456
5,1202826,zaccaro,433
6,622729,finlay,426
7,7092593,flafuji,418
8,7413986,yujimizobuchi,404
9,1177718,zzzwilliam2003,395


In [17]:
# The session: users with most team memberships
result = session.query(User.id, User.username, func.count(TeamMembership.id).label("number of team memberships")).filter(User.id==TeamMembership.userid).group_by(User.id).order_by(desc("number of team memberships")).limit(10).all()
print(result)

[(5030586, 'wchenfla', 1651), (929585, 'plarmuseau', 823), (3258, 'wcukierski', 534), (993, 'benhamner', 476), (18463, 'mathurinache', 456), (1202826, 'zaccaro', 433), (622729, 'finlay', 426), (7092593, 'flafuji', 418), (7413986, 'yujimizobuchi', 404), (1177718, 'zzzwilliam2003', 395)]


As you can see, both methods yield the same results. Wchenfla is the clear winner.

## More complex queries

Now I am going to build more complex queries. This time, I will only use normal SQL language and queries in order to get the data out of the database. Let's ask some questions about and build some queries on the forums of Kaggle.

### The first forum message for the top 10 forum messengers 

Suppose that we are interested in the first ever message that is being posted by the most heavy users. That is, we would to identify the top 10 messengers, and then select their first forum message. 

In [18]:
query = """WITH added_row_number AS (
                SELECT *, ROW_NUMBER() OVER(PARTITION BY postuserid ORDER BY postdate) AS row_number
                FROM forummessages
                ),
            
                first_message AS (
                SELECT *
                FROM added_row_number
                WHERE row_number = 1
                ),
            
                top10 AS(
                SELECT forummessages.postuserid, users.username, COUNT(forummessages.id) AS number_of_forum_messages
                FROM forummessages
                INNER JOIN users ON forummessages.postuserid=users.id
                GROUP BY forummessages.postuserid
                ORDER BY "number_of_forum_messages" DESC
                LIMIT 10)
            
            SELECT top10.postuserid, top10.username, top10.number_of_forum_messages, first_message.message
            FROM top10
            INNER JOIN first_message ON top10.postuserid=first_message.postuserid;
"""



retrieved = pd.read_sql(query, engine)

retrieved

,postuserid,username,number_of_forum_messages,message
0,3012786,mpwolke,12896,"<p>I ended my first exercise. There is no ""cl..."
1,75976,cpmpml,8845,"<p>You may find this useful: <a href=""https://..."
2,1723677,cdeotte,6056,<p>Awesome. Thanks.</p>
3,113660,hengck23,4402,"<p>Hi,</p>\n\n<p>Although the sample submissio..."
4,3258,wcukierski,2599,Here's a question you may or may not want to a...
5,1314380,paultimothymooney,2412,<p>You did a great job with feature engineerin...
6,2332181,frtgnn,2379,"<p>wow, cool! </p>"
7,59561,inversion,2328,<p>Holy Smokes! That's going to be a challengi...
8,898111,sheriytm,2249,<p>When I run the following command to verify ...
9,1162990,rtatman,2189,<p>Really neat work! I especially like the tim...


### Which forum topics get the most votes from the 20 percent most frequent forum messengers?

Suppose that we are interested in which forum topics get the most votes. This could be a simple query. But suppose that for some reason we only want to know this for the most frequent forum messengers. Let's build a query that finds the data for the 20 percent most frequent forum messengers.

In [19]:
query = """ WITH number_of_messages AS(
                SELECT postuserid, COUNT(*) AS number_of_messages
                FROM forummessages
                GROUP BY postuserid),
        
                all_users AS(
                SELECT users.id, number_of_messages.postuserid, CASE WHEN number_of_messages.number_of_messages IS NULL THEN 0 ELSE number_of_messages.number_of_messages END AS number_of_messages
                FROM users
                LEFT JOIN number_of_messages ON users.id=number_of_messages.postuserid),
                
                all_users2 AS(
                SELECT id, number_of_messages, PERCENT_RANK() OVER(ORDER BY number_of_messages) AS message_rank
                FROM all_users),
                
                most_frequent_messengers AS(
                SELECT * 
                FROM all_users2
                WHERE message_rank > 0.8)
                
            SELECT forumtopics.forumid, forumtopics.title, COUNT(forumtopics.forumid) AS "number of forum message votes by top forum messengers"
            FROM most_frequent_messengers
            INNER JOIN forummessagevotes ON forummessagevotes.fromuserid=most_frequent_messengers.id
            INNER JOIN forummessages ON forummessages.id=forummessagevotes.forummessageid
            INNER JOIN forumtopics ON forumtopics.id=forummessages.forumtopicid
            GROUP BY forumtopics.forumid
            ORDER BY "number of forum message votes by top forum messengers" DESC
            LIMIT 10;
"""

retrieved = pd.read_sql(query, engine)

retrieved

,forumid,title,number of forum message votes by top forum messengers
0,16860,Testing comments,320049
1,15,Tips for beautiful math posts,89813
2,208,New to Data Science (formally),86089
3,2239,ERROR FOR TITANIC DATASET,39600
4,130117,Interesting that ALL the values are masked here.,25066
5,693509,Welcome!,23350
6,275877,External Data Disclosure Thread,22856
7,809,Kaggle Scripts Product Feedback,21823
8,17686,Welcome to The Kaggle Learn Forum,20787
9,872112,Connectivity Map,20317


### Top 10 users with longest time intervals regarding forum message posts

Now suppose that we want to identify those users that haven't posted for a very long time. Let's find the top 10 users with the longest time intervals regarding forum message posts!

In [20]:
query = """WITH lag_date AS(
                SELECT postuserid, postdate, LAG(postdate) OVER(PARTITION BY postuserid ORDER BY postuserid, postdate) AS lag_postdate
                FROM forummessages),
                
            date_dif AS(            
                SELECT postuserid, Cast ((JulianDay(postdate) - JulianDay(lag_postdate)) AS Integer) AS date_dif
                FROM lag_date
                WHERE lag_postdate IS NOT NULL)
            
            SELECT date_dif.postuserid, users.username, MAX(date_dif.date_dif) AS "interval in days"
            FROM date_dif
            INNER JOIN users ON users.id=date_dif.postuserid
            GROUP BY users.id
            ORDER BY "interval in days" DESC
            LIMIT 10;
            
"""
retrieved = pd.read_sql(query, engine)

retrieved

,postuserid,username,interval in days
0,3991,duck3991,3904
1,4036,dane4036,3904
2,7980,muhammad,3771
3,1927,bernhard1,3654
4,14977,silentpartnr,3648
5,17368,tonyjung,3594
6,1388,jay1388,3540
7,4497,igor14497,3524
8,6380,xiaonanji,3496
9,25788,romanos0,3478


## Conclusion

In this project I build a database and then used both the SQLAlchemy ORM Session and normal SQL queries to find data. Of course, there are much more queries that can be built, as the database is very rich.